# MoBioFP - Feature Extraction and Matching

In [ ]:
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

## Define global constants

In [ ]:
# Processed Sample Directory (results of the pre-processing using figertip object detection)
SAMPLE_DIR = "../data/processed/samples/detection"

# # Processed Sample Directory (results of the pre-processing using figertip semantic segmentation)
# SAMPLE_DIR = "../data/processed/samples/segmentation"

## Define Utility Functions

In [ ]:
def show_images(images, titles, cmap="gray", show_axis=False, fig_size=10, sup_title=None, limit=8):
    assert (titles is None) or (len(images) == len(titles))

    # Limit the number of images and titles
    images = images[:limit]
    titles = titles[:limit] if titles else None

    num_images = len(images)
    num_cols = 4
    num_rows = math.ceil(num_images / num_cols)

    fig_height = fig_size * (num_rows / num_cols) * 1.5
    _, axes = plt.subplots(
        num_rows, num_cols, figsize=(fig_size, fig_height), constrained_layout=True
    )
    axes = axes.ravel()

    if sup_title:
        plt.suptitle(sup_title, fontsize=24)

    for idx, (image, title) in enumerate(zip(images, titles)):
        axes[idx].imshow(image, cmap=cmap)
        axes[idx].set_title(title, fontsize=12)
        axes[idx].axis("on" if show_axis else "off")

    # Hide the remaining subplots
    for idx in range(num_images, num_cols * num_rows):
        axes[idx].axis("off")


plt.show()


def split_probe_gallery(images, images_titles):
    NUM_TEMPLATES = 4  # Number of templates for each subject
    subjects = {}

    for title in images_titles:
        subject_id, _, _, _ = title.split("_")

        if subject_id in subjects:
            subjects[subject_id] += 1
        else:
            subjects[subject_id] = 1

    for subject_id in list(subjects.keys()):
        # Remove the subject if it does not have the required number of templates
        if subjects[subject_id] != NUM_TEMPLATES:
            del subjects[subject_id]

    probe = []
    gallery = []

    # 2. Split the images into probe and gallery by using the filtered subjects dictionary
    for image, title in zip(images, images_titles):
        subject_id, illumination, finger_id, background = title.split("_")

        if subject_id in subjects:
            if illumination == "o" and background == "n":
                probe.append((image, title))
            elif illumination == "i" and background == "w":
                gallery.append((image, title))
            else:
                print(f"Skipping template: {title}")

    assert len(probe) == len(gallery)

    # 3. Sort the probe and gallery images based on the subject id
    probe = sorted(probe, key=lambda x: x[1])
    gallery = sorted(gallery, key=lambda x: x[1])

    return probe, gallery


def bf_matcher(img1, kp1, desc1, img2, kp2, desc2):
    # Create a brute-force matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Perform the matching between the two descriptor sets
    matches = bf.match(desc1, desc2)

    # Sort the matches based on distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Draw the matches
    matches_img = cv2.drawMatches(img1, kp1, img2, kp2, matches, None)

    # Compute median distance
    distance = np.median([m.distance for m in matches])

    return distance, matches_img


def flann_matcher(img1, kp1, desc1, img2, kp2, desc2):
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm=FLANN_INDEX_LSH, table_number=6, key_size=12, multi_probe_level=1)
    search_params = dict(checks=50)

    # Create a FLANN matcher object
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desc1, desc2, k=2)

    # Apply ratio test
    good_matches = []
    for match in matches:
        if len(match) >= 2:
            m, n = match
            if m.distance < 0.75 * n.distance:
                good_matches.append(m)

    # Draw the good matches
    matches_img = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None)

    # Compute median distance
    if good_matches:
        distance = np.median([m.distance for m in good_matches])
        # Check if distance is nan or inf, if so, set it to a large number
        if np.isnan(distance) or np.isinf(distance):
            distance = float("inf")
    else:
        distance = float("inf")

    return distance, matches_img


def distance_matrix(
    probe_images,
    probe_titles,
    probe_orb_keypoints,
    probe_orb_descriptors,
    gallery_images,
    gallery_titles,
    gallery_orb_keypoints,
    gallery_orb_descriptors,
    matcher=flann_matcher,
    matched_img=False,
):
    distances = {p: [] for p in probe_titles}

    for p_im, p_title, p_kp, p_desc in zip(
        probe_images, probe_titles, probe_orb_keypoints, probe_orb_descriptors
    ):
        for g_im, g_title, g_kp, g_desc in zip(
            gallery_images, gallery_titles, gallery_orb_keypoints, gallery_orb_descriptors
        ):
            if not check(p_title, g_title):
                continue

            values = {}
            values[g_title] = {}
            distance, matches_img = matcher(p_im, p_kp, p_desc, g_im, g_kp, g_desc)
            if np.isnan(distance) or np.isinf(distance):
                distance = -1

            values[g_title]["distance"] = distance

            if matched_img:
                values[g_title]["matches_img"] = matches_img
            distances[p_title].append(values)

    return distances


def check(probe_title, gallery_title):
    _, probe_illumination, probe_finger, probe_background = probe_title.split("_")
    _, gallery_illumination, gallery_finger, gallery_background = gallery_title.split("_")

    return (
        probe_illumination == "o"
        and gallery_illumination == "i"
        and probe_background == "n"
        and gallery_background == "w"
        and probe_finger == gallery_finger
    )

## Read Fingerprint Images

In [ ]:
images_paths = list(Path(SAMPLE_DIR).rglob("*.png"))
images = []
images_titles = []

for p in images_paths:
    img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
    images.append(img)
    images_titles.append(p.stem)

## Split Fingerprint into Probe (NO) and Gallery (WI)

In [ ]:
probe, gallery = split_probe_gallery(images, images_titles)
probe_images, probe_titles = zip(*probe)
gallery_images, gallery_titles = zip(*gallery)

print(f"Probe (NO) # templates (1 subject, 2 templates): {len(probe)}")
print(f"Gallery (WI) # subjects (1 subject, 2 templates): {len(gallery)}")

# Uncomment these lines to apply Zhang-Suen thinning algorithm to the images
# probe_images = [cv2.ximgproc.thinning(img) for img in probe_images]
# gallery_images = [cv2.ximgproc.thinning(img) for img in gallery_images]

In [ ]:
show_images(probe_images, probe_titles, fig_size=8, sup_title="Probe (NO) Fingerprint Images")
show_images(gallery_images, gallery_titles, fig_size=8, sup_title="Gallery (WI) Fingerprint Images")

## Extract Features

### 1. ORB (Oriented FAST and Rotated BRIEF)

REF: https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html

In [ ]:
# Initialize the ORB detector
orb = cv2.ORB_create()

# Find the keypoints and descriptors for the probe and gallery images
probe_orb_keypoints, probe_orb_descriptors = zip(
    *[orb.detectAndCompute(image, None) for image in probe_images]
)
gallery_orb_keypoints, gallery_orb_descriptors = zip(
    *[orb.detectAndCompute(image, None) for image in gallery_images]
)

# Show keypoints for each probe and images
probe_orb_keypoints_images = [
    imkpts(image, keypoints) for image, keypoints in zip(probe_images, probe_orb_keypoints)
]
gallery_orb_keypoints_images = [
    imkpts(image, keypoints) for image, keypoints in zip(gallery_images, gallery_orb_keypoints)
]

show_images(
    probe_orb_keypoints_images, probe_titles, fig_size=15, sup_title="Probe (NO) ORB Keypoints"
)
show_images(
    gallery_orb_keypoints_images,
    gallery_titles,
    fig_size=15,
    sup_title="Gallery (WI) ORB Keypoints",
)

#### Compute Distance Matrix

In [ ]:
distances = distance_matrix(
    probe_images,
    probe_titles,
    probe_orb_keypoints,
    probe_orb_descriptors,
    gallery_images,
    gallery_titles,
    gallery_orb_keypoints,
    gallery_orb_descriptors,
    matched_img=True,
)